In [1]:
from os import listdir
from os.path import isfile, join, isdir
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model
from cv2 import cv2
import time

In [2]:
SINGLE_TEST_PATH = "F:\\Research database\\UCSD_Anomaly_Dataset.v1p2\\UCSDped1\\Test\\Test014"

In [3]:
all_cv2_image = list()
def get_single_test(path):
    sz = 200
    test = np.zeros(shape=(sz, 256, 256, 1))
    cnt = 0
    for f in sorted(listdir(path)):
        if str(join(path, f))[-3:] == "tif":
            img = Image.open(join(path, f)).resize((256, 256))
            opencvImage = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
            all_cv2_image.append(cv2.resize(opencvImage, (500, 300)))
            img = np.array(img, dtype=np.float32) / 255.0
            test[cnt, :, :, 0] = img
            cnt = cnt + 1
    return test

In [4]:

model = load_model('model.h5')
test = get_single_test(SINGLE_TEST_PATH)
sz = test.shape[0] - 5
sequences = np.zeros((sz, 5, 256, 256, 1))
# apply the sliding window technique to get the sequences
for i in range(0, sz):
    clip = np.zeros((5, 256, 256, 1))
    for j in range(0, 5):
        clip[j] = test[i + j, :, :, :]
    sequences[i] = clip
reconstructed_sequences = model.predict(sequences,batch_size=10)
sequences_reconstruction_cost = np.array([np.linalg.norm(np.subtract(sequences[i],reconstructed_sequences[i])) for i in range(0,sz)])
sa = (sequences_reconstruction_cost - np.min(sequences_reconstruction_cost)) / np.max(sequences_reconstruction_cost)
sr = 1.0 - sa

In [5]:
# sr

In [6]:
# anomaly_mean = np.mean(sr)
anomaly_mean = 0.92

##  anomaly visualization

In [11]:
for i in range(len(sr)):
    if sr[i] < anomaly_mean:
        new_image = cv2.putText(img = all_cv2_image[i],text = "Anomaly Detected", org = (50, 70),
                                fontFace = cv2.FONT_HERSHEY_DUPLEX,fontScale = 1.0,
                                color = (0, 255, 0),thickness = 2)
    cv2.imshow('Video',all_cv2_image[i])
    time.sleep(0.09)
    cv2.waitKey(1)
cv2.destroyAllWindows()